In [1]:
"""
CODE to generate artery graphs.
"""
import json
import time

import matlab.engine
import networkx as nx
import sys
sys.path.append("../../")
import pandas as pd
import webcolors
import numpy as np
import os
import math
import cv2
import matplotlib.pyplot as plt
import pickle
import argparse

from scipy.interpolate import interp1d
from tqdm import tqdm
from skimage import measure
from glob import glob

from core.utils.module import Node, VesselSegment
from core.graph_generation import visualization as visualizer
from PIL import Image


import constants

In [9]:
from data_processing import diameter_map, get_spacing

In [3]:
args = {"data_dir": '/media/zhaochen/data/fluoro_gnn/data/data_no_semantic',
        "save_dir": '/media/zhaochen/data/fluoro_gnn/data/data_no_semantic/processed',
        "centerline_length_threshold": 10,
        "radius_threshold": 1.8}

class obj:
    def __init__(self, dict1):
        self.__dict__.update(dict1)
        
def dict2obj(dict1):
    return json.loads(json.dumps(dict1), object_hook=obj)

args = dict2obj(args)
    
eng = matlab.engine.start_matlab()

In [5]:
patient_name = '7_LCA_RAO2'
original_file_path = os.path.join(args.data_dir, 'training/{}.png'.format(patient_name))
binary_file_path = os.path.join(args.data_dir, 'label/{}.png'.format(patient_name))
semantic_files_path = os.path.join(args.data_dir, 'image', patient_name)
dist_map = diameter_map(os.path.join(args.data_dir, f'label/{patient_name}.png'), eng)

In [16]:
vessel_objects = pickle.load(open(os.path.join(args.save_dir, f"{patient_name}_ori.pkl"), "rb"))
print(len(vessel_objects))
g = pickle.load(open(os.path.join(args.save_dir, f"{patient_name}_graph.pkl"), "rb"))
nodes = pickle.load(open(os.path.join(args.save_dir, f"{patient_name}_nodes.pkl"), "rb"))

26


In [17]:
# def filter_vessel(vessel_objs: list, g: nx.Graph, nodes: list, dist_map,
#                   patient_name, csv_path, radius_threshold, centerline_length_threshold):

#filter vessel
v_objs = []
pixel_spacing = get_spacing(os.path.join(args.data_dir, "pixel_spacing.csv"), patient_name)

for idx, vessel_obj in enumerate(vessel_objects):
    segment_dist_map = vessel_obj.vessel_centerline * dist_map
    # assign distance map to each vessel object
    vessel_obj.vessel_centerline_dist = segment_dist_map
    # average_dist = np.mean(segment_dist_map[segment_dist_map > 0])
    mean_dist = np.mean(segment_dist_map[segment_dist_map > 0])
    min_dist = np.min(segment_dist_map[segment_dist_map > 0])
    max_dist = np.max(segment_dist_map[segment_dist_map > 0])

    if max_dist * pixel_spacing >= args.radius_threshold and \
        np.sum(vessel_obj.vessel_centerline) > args.centerline_length_threshold:
        v_objs.append(vessel_obj)
    else:
        # remove vessel_object
        if vessel_obj.node1.degree == 1:
            # remove edge with end points: in implementation, remove node will automatically remove edge
            g.remove_node(nodes.index(vessel_obj.node1))
        elif vessel_obj.node2.degree == 1:
            # remove node with end points: in implementation, remove node will automatically remove edge
            g.remove_node(nodes.index(vessel_obj.node2))
        else:
            # remove edge without end points
            g.remove_edge(nodes.index(vessel_obj.node1), nodes.index(vessel_obj.node2))
            if not g.is_connected():
                g.add_edge(nodes.index(vessel_obj.node1), nodes.index(vessel_obj.node2))
                v_objs.append(vessel_obj)

ValueError: <core.utils.module.Node object at 0x7f2ae0aac1d0> is not in list